In [1]:
import pandas as pd
import plotly.express as px

# Results postprocessing

In [4]:
community = 'duluth' # TODO change community name
#Total_dwelling_unit_count = 39762  # TODO supplied by the community

up00 = pd.read_parquet(f'data_/community_building_samples_with_upgrade_cost_and_bill/{community}/up00__{community}.parquet', engine='pyarrow')


## Housing characterization distribution

### 1. Vintage

In [8]:
# regroup input data from Jes
vintage_list = ['<1940', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s']
vintage_figure = ['Pre 1940', '1940-1979', 'Post 1980']
vintage_percentage = []
vintage_percentage_figure = []
for vintage in vintage_list:
    temp= up00.loc[up00['build_existing_model.vintage'] == vintage]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    vintage_percentage.append(temp_count)
    
vintage_percentage_figure.append(vintage_percentage[0])
temp = vintage_percentage[1] + vintage_percentage[2] + vintage_percentage[3]+ vintage_percentage[4]
vintage_percentage_figure.append(temp)
temp = vintage_percentage[5] + vintage_percentage[6] + vintage_percentage[7]+ vintage_percentage[8]
vintage_percentage_figure.append(temp)

fig = px.bar(x=vintage_figure, y=vintage_percentage_figure, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=250, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  xaxis_title="Construction Year",
                  yaxis_title="Percentage of Community Housing",
                  yaxis_range=[0,0.4])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image("vintage.svg")

### 2. Building type

In [9]:
building_type_list = ['Single-Family Detached', 
                      'Single-Family Attached', 
                      'Multi-Family with 2 - 4 Units', 
                      'Multi-Family with 5+ Units',
                      'Mobile Home']
building_type_figure = ['Single-Family Detached', 'Single-Family Attached', 'Multi-Family 2-4 units', 'Multi-Family 5+ units', 'Mobile Home']
building_type_percentage = []
for building_type in building_type_list:
    temp = up00.loc[up00['build_existing_model.geometry_building_type_recs'] == building_type]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    building_type_percentage.append(temp_count)
    
fig = px.bar(x=building_type_figure, y=building_type_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=320, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  xaxis_title="Building Type",
                  yaxis_title="Percentage of Community Housing",
                 yaxis_range=[0,0.8])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image("building_type.svg")

### 3. Heating fuel

In [10]:
heating_fuel_list = ['electricity', 'natural_gas', 'propane', 'fuel_oil']
heating_fuel_figure = ['Electricity', 'Natural Gas', 'Propane', 'Fuel Oil']
heating_fuel_consumption = []
for heating_fuel in heating_fuel_list:
    temp = up00[f'report_simulation_output.end_use_{heating_fuel}_heating_m_btu'].sum() 
    heating_fuel_consumption.append(temp)
    
total_energy = sum(heating_fuel_consumption)
heating_fuel_percentage = heating_fuel_consumption/total_energy

fig = px.bar(x=heating_fuel_figure, y=heating_fuel_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=290, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  xaxis_title="Heating Fuel",
                  yaxis_title="Percentage of Community Housing",
                 yaxis_range=[0,0.8])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image("heating_fuel.svg")

### 4. Cooling type

In [11]:
cooling_type_list = list(up00['build_existing_model.hvac_cooling_type'].unique())
cooling_type_percentage = []
for cooling_type in cooling_type_list:
    temp = up00.loc[up00['build_existing_model.hvac_cooling_type'] == cooling_type]
    temp_count =temp['sample_weight'].sum()/Total_dwelling_unit_count
    cooling_type_percentage.append(temp_count)
    
fig = px.bar(x=cooling_type_list, y=cooling_type_percentage, text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=290, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  xaxis_title="Cooling Type",
                  yaxis_title="Percentage of Community Housing",
                  yaxis_range=[0,0.6])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image("cooling_type.svg")

### 5. Envelope label

Envelope label criteria

In [13]:
# TODO, define the breakpoint
insulation_wall_r1 = 8
insulation_wall_r2 = 13
insulation_attic_r1 = 25
insulation_attic_r2 = 38
infiltration1 = 5
infiltration2 = 10

In [14]:
up00.loc[up00['build_existing_model.insulation_wall'] == 'Wood Stud, Uninsulated', 'build_existing_model.wall_assembly_r'] = 3.4
up00.loc[up00['build_existing_model.insulation_wall'] == 'Wood Stud, R-7', 'build_existing_model.wall_assembly_r'] = 8.7
up00.loc[up00['build_existing_model.insulation_wall'] == 'Wood Stud, R-11', 'build_existing_model.wall_assembly_r'] = 10.3
up00.loc[up00['build_existing_model.insulation_wall'] == 'Wood Stud, R-15', 'build_existing_model.wall_assembly_r'] = 12.1
up00.loc[up00['build_existing_model.insulation_wall'] == 'Wood Stud, R-19', 'build_existing_model.wall_assembly_r'] = 15.4
up00.loc[up00['build_existing_model.insulation_wall'] == 'CMU, 6-in Hollow, Uninsulated', 'build_existing_model.wall_assembly_r'] = 4
up00.loc[up00['build_existing_model.insulation_wall'] == 'CMU, 6-in Hollow, R-7', 'build_existing_model.wall_assembly_r'] = 9.4
up00.loc[up00['build_existing_model.insulation_wall'] == 'CMU, 6-in Hollow, R-11', 'build_existing_model.wall_assembly_r'] = 12.4
up00.loc[up00['build_existing_model.insulation_wall'] == 'CMU, 6-in Hollow, R-15', 'build_existing_model.wall_assembly_r'] = 15
up00.loc[up00['build_existing_model.insulation_wall'] == 'CMU, 6-in Hollow, R-19', 'build_existing_model.wall_assembly_r'] = 17.4
up00.loc[up00['build_existing_model.insulation_wall'] == 'Brick, 12-in, 3-wythe, Uninsulated', 'build_existing_model.wall_assembly_r'] = 4.9
up00.loc[up00['build_existing_model.insulation_wall'] == 'Brick, 12-in, 3-wythe, R-7', 'build_existing_model.wall_assembly_r'] = 10.3
up00.loc[up00['build_existing_model.insulation_wall'] == 'Brick, 12-in, 3-wythe, R-11', 'build_existing_model.wall_assembly_r'] = 13.3
up00.loc[up00['build_existing_model.insulation_wall'] == 'Brick, 12-in, 3-wythe, R-15', 'build_existing_model.wall_assembly_r'] = 15.9
up00.loc[up00['build_existing_model.insulation_wall'] == 'Brick, 12-in, 3-wythe, R-19', 'build_existing_model.wall_assembly_r'] = 18.3

up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Aluminum, Light', 'build_existing_model.wall_exterior_finish_r'] = 0.6
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Brick, Light', 'build_existing_model.wall_exterior_finish_r'] = 0.7
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Brick, Medium/Dark', 'build_existing_model.wall_exterior_finish_r'] = 0.7
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Fiber-Cement, Light', 'build_existing_model.wall_exterior_finish_r'] = 0.2
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Shingle, Composition, Medium', 'build_existing_model.wall_exterior_finish_r'] = 0.6
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Shingle, Asbestos, Medium', 'build_existing_model.wall_exterior_finish_r'] = 0.6
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Stucco, Light', 'build_existing_model.wall_exterior_finish_r'] = 0.2
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Stucco, Medium/Dark', 'build_existing_model.wall_exterior_finish_r'] = 0.2
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Vinyl, Light', 'build_existing_model.wall_exterior_finish_r'] = 0.6
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'Wood, Medium/Dark', 'build_existing_model.wall_exterior_finish_r'] = 1.4
up00.loc[up00['build_existing_model.geometry_wall_exterior_finish'] == 'None', 'build_existing_model.wall_exterior_finish_r'] = 0

up00.loc[up00['build_existing_model.insulation_ceiling'] == 'None', 'build_existing_model.ceiling_assembly_r'] = 0
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'Uninsulated', 'build_existing_model.ceiling_assembly_r'] = 2.1
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-7', 'build_existing_model.ceiling_assembly_r'] = 8.7
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-13', 'build_existing_model.ceiling_assembly_r'] = 14.6
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-19', 'build_existing_model.ceiling_assembly_r'] = 20.6
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-30', 'build_existing_model.ceiling_assembly_r'] = 31.6
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-38', 'build_existing_model.ceiling_assembly_r'] = 39.6
up00.loc[up00['build_existing_model.insulation_ceiling'] == 'R-49', 'build_existing_model.ceiling_assembly_r'] = 50.6

up00.loc[up00['build_existing_model.insulation_roof'] == 'Unfinished, Uninsulated', 'build_existing_model.roof_assembly_r'] = 2.3
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, Uninsulated', 'build_existing_model.roof_assembly_r'] = 3.7
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-7', 'build_existing_model.roof_assembly_r'] = 10.2
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-13', 'build_existing_model.roof_assembly_r'] = 14.3
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-19', 'build_existing_model.roof_assembly_r'] = 21.2
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-30', 'build_existing_model.roof_assembly_r'] = 29.7
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-38', 'build_existing_model.roof_assembly_r'] = 36.5
up00.loc[up00['build_existing_model.insulation_roof'] == 'Finished, R-49', 'build_existing_model.roof_assembly_r'] = 47.0

up00['build_existing_model.insulation_wall_r'] = up00['build_existing_model.wall_assembly_r'].fillna(0) + up00['build_existing_model.wall_exterior_finish_r'].fillna(0)
up00['build_existing_model.insulation_attic_r'] = up00['build_existing_model.ceiling_assembly_r'].fillna(0) + up00['build_existing_model.roof_assembly_r'].fillna(0)

up00['build_existing_model.infiltration_new'] = up00['build_existing_model.infiltration'].str.replace(' ACH50', '')
up00['build_existing_model.infiltration_new'] = up00['build_existing_model.infiltration_new'].astype(float)

In [15]:
good = up00.loc[(up00['build_existing_model.insulation_wall_r'] > insulation_wall_r2) 
                & (up00['build_existing_model.insulation_attic_r'] > insulation_attic_r2) 
                & (up00['build_existing_model.infiltration_new'] > infiltration2)]
inadequate_and_good = up00.loc[(up00['build_existing_model.insulation_wall_r'] > insulation_wall_r1) 
                & (up00['build_existing_model.insulation_attic_r'] > insulation_attic_r1) 
                & (up00['build_existing_model.infiltration_new'] > infiltration1)]

total_count = len(up00)
good_count = len(good)/total_count
inadequate_count = (len(inadequate_and_good) - good_count)/total_count
poor_count = 1 - good_count - inadequate_count

fig = px.bar(x=['Good Envelope', 'Inadequate Envelope', 'Poor Envelope '], y=[good_count, inadequate_count, poor_count], text_auto=True)
fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
fig.update_layout(autosize=False, width=250, height=300, 
                  font=dict(size=10), 
                  plot_bgcolor='rgba(0, 0, 0, 0)', 
                  xaxis_title="Envelope Status",
                  yaxis_title="Percentage of Community Housing",
                  yaxis_range=[0,0.67])
fig.layout.yaxis.tickformat = ',.0%'
fig.update_xaxes(tickangle=45)
fig.show()
fig.write_image("envelope_lable.svg")